In [1]:
import ScraperFC as sfc
import pandas as pd
import os

In [2]:
sofascore = sfc.Sofascore()

In [3]:
year = "2025"
liga = 'Liga 1 Peru'

In [ ]:
comps = {
    # Copas internaciones de países (hombres)
    'World Cup': 16, 'Euros': 1, 'Gold Cup': 140, 'Copa America': 133,
    # Copas Continentales de clubes
    'Champions League': 7, 'Europa League': 679, 'Europa Conference League': 17015, 
    "Copa Libertadores": 384, 'Copa Sudamericana': 480,
    # Ligas europeas
    'EPL': 17, 'La Liga': 8, 'Bundesliga': 35, 'Serie A': 23, 'Ligue 1': 34, 'Turkish Super Lig': 52,
    # Ligas de AMERICA DEL SUR
    'Argentina Liga Profesional': 155, 'Argentina Copa de la Liga Profesional': 13475,
    'Liga 1 Peru': 406,  'Chile Primera Division': 11653, 'Venezuela Primera Division': 231,
    'Uruguay Primera Division': 278 , 'Ecuador LigaPro':240, 'Brasileirão Série A':325,
    # Ligas de USA
    'MLS': 242, 'USL Championship': 13363, 'USL1': 13362, 'USL2': 13546,
    # Ligas de Mexico
    'Mexico LigaMX Apertura':11621, 'LigaMX Clausura': 11620,
    # Ligas de Medio Oriente
    "Saudi Pro League": 955,
    # Copas internacionales de mujeres
    "Women's World Cup": 290
}

In [4]:
partidos = sofascore.get_match_dicts(year=year, league=liga)

Running


In [5]:
rows = []

for partido in partidos:
    match_id = str(partido.get('id', ''))

    # URL sin request extra
    home_slug = partido.get('homeTeam', {}).get('slug', '')
    away_slug = partido.get('awayTeam', {}).get('slug', '')
    custom_id = partido.get('customId', '')
    match_url = f"https://www.sofascore.com/{home_slug}-{away_slug}/{custom_id}#id:{match_id}"

    # Campos rápidos (del listado)
    home_name   = partido.get('homeTeam', {}).get('name')
    home_id     = partido.get('homeTeam', {}).get('id')
    away_name   = partido.get('awayTeam', {}).get('name')
    away_id     = partido.get('awayTeam', {}).get('id')
    home_score  = partido.get('homeScore', {}).get('current')
    away_score  = partido.get('awayScore', {}).get('current')

    hc = partido.get('homeTeam', {}).get('teamColors', {}) or {}
    ac = partido.get('awayTeam', {}).get('teamColors', {}) or {}
    home_colors = f"Primary: {hc.get('primary','')}, Secondary: {hc.get('secondary','')}"
    away_colors = f"Primary: {ac.get('primary','')}, Secondary: {ac.get('secondary','')}"

    tournament   = partido.get('tournament', {}).get('name')
    round_number = partido.get('roundInfo', {}).get('round')
    season       = partido.get('season', {}).get('name')

    # ---- Detalle (1 request por partido) ----
    md = sofascore.get_match_dict(match_id)

    ht = md.get('homeTeam', {}) or {}
    at = md.get('awayTeam', {}) or {}
    venue_dict = (md.get('venue') or ht.get('venue') or at.get('venue') or {})  # fallback

    # Asignaciones seguras
    manager_home = (ht.get('manager') or {}).get('name')
    manager_away = (at.get('manager') or {}).get('name')

    hs = md.get('homeScore', {}) or {}
    as_ = md.get('awayScore', {}) or {}

    goles_1T_home = hs.get('period1')
    goles_2T_home = hs.get('period2')
    goles_1T_away = as_.get('period1')
    goles_2T_away = as_.get('period2')

    disp_home = hs.get('display')
    disp_away = as_.get('display')
    resultado_final = f"{disp_home} - {disp_away}" if disp_home is not None and disp_away is not None else None

    estadio = venue_dict.get('name')
    ciudad  = (venue_dict.get('city') or {}).get('name')
    fecha   = pd.to_datetime(md.get('startTimestamp'), unit='s', errors='coerce')
    arbitro = (md.get('referee') or {}).get('name')

    # Construcción de la fila (sin .update con getters anidados)
    row = {
        'match_id'          : match_id,
        'match_url'         : match_url,
        'home'              : home_name,
        'home_id'           : home_id,
        'home_score'        : home_score,
        'away'              : away_name,
        'away_id'           : away_id,
        'away_score'        : away_score,
        'home_team_colors'  : home_colors,
        'away_team_colors'  : away_colors,
        'tournament'        : tournament,
        'round_number'      : round_number,
        'season'            : season,

        # resumen detallado
        'manager_home'      : manager_home,
        'manager_away'      : manager_away,
        'goles_1T_home'     : goles_1T_home,
        'goles_2T_home'     : goles_2T_home,
        'goles_1T_away'     : goles_1T_away,
        'goles_2T_away'     : goles_2T_away,
        'resultado_final'   : resultado_final,
        'estadio'           : estadio,
        'ciudad'            : ciudad,
        'fecha'             : fecha,
        'arbitro'           : arbitro,
    }

    rows.append(row)

# Tabla final en Jupyter
df_matches = pd.DataFrame(rows).reset_index(drop=True)
df_matches

,match_id,match_url,home,home_id,home_score,away,away_id,away_score,home_team_colors,away_team_colors,...,manager_away,goles_1T_home,goles_2T_home,goles_1T_away,goles_2T_away,resultado_final,estadio,ciudad,fecha,arbitro
0,14107749,https://www.sofascore.com/los-chankas-cyc-come...,Los Chankas CYC,252254,3.0,Comerciantes Unidos,213609,2.0,"Primary: #ffffff, Secondary: #e20e0e","Primary: #ffff00, Secondary: #ffff00",...,Claudio Biaggio,2.0,1.0,2.0,0.0,3 - 2,Estadio Los Chankas,Andahuaylas,2025-08-01 18:00:00,Jonathan Zamora
1,14107752,https://www.sofascore.com/alianza-atletico-de-...,Alianza Atlético de Sullana,2307,0.0,Melgar,2308,0.0,"Primary: #ffffff, Secondary: #6600ff","Primary: #000000, Secondary: #ff0000",...,Juan Maximo Reynoso,0.0,0.0,0.0,0.0,0 - 0,Estadio Campeones del 36,Sullana,2025-08-01 20:15:00,Julio Cesar Quiroz
2,14107753,https://www.sofascore.com/deportivo-garcilaso-...,Deportivo Garcilaso,458584,2.0,Cienciano,2301,1.0,"Primary: #374df5, Secondary: #374df5","Primary: #ff0000, Secondary: #ff0000",...,Carlos Desio,1.0,1.0,1.0,0.0,2 - 1,Estadio Inca Garcilaso de la Vega,Cusco,2025-08-02 00:00:00,Daniel Ureta
3,14186416,https://www.sofascore.com/sport-huancayo-ayacu...,Sport Huancayo,33895,3.0,Ayacucho,33894,1.0,"Primary: #ff0000, Secondary: #ffff00","Primary: #ffffff, Secondary: #000000",...,Josué Castells,2.0,1.0,1.0,0.0,3 - 1,Estadio Huancayo,Huancayo,2025-08-03 20:00:00,Sebastian Lozano
4,14186417,https://www.sofascore.com/universidad-tecnica-...,Universidad Técnica de Cajamarca,87854,1.0,Cusco FC,63760,2.0,"Primary: #ffffff, Secondary: #ffffff","Primary: #00ffff, Secondary: #33ffff",...,Miguel Rondelli,1.0,0.0,1.0,1.0,1 - 2,Estadio Heroes de San Ramon,Cajamarca,2025-08-04 18:00:00,Jonathan Zamora
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,13387757,https://www.sofascore.com/club-sporting-crista...,Club Sporting Cristal,2302,2.0,Sport Boys,2312,1.0,"Primary: #33ccff, Secondary: #ffffff","Primary: #ffc0db, Secondary: #000000",...,Guillermo Vasquez,1.0,1.0,0.0,1.0,2 - 1,Estadio Alberto Gallardo,Lima,2025-02-16 16:00:00,Daniel Ureta
225,13387753,https://www.sofascore.com/adc-juan-pablo-ii-sp...,ADC Juan Pablo II,511206,0.0,Sport Huancayo,33895,1.0,"Primary: #374df5, Secondary: #374df5","Primary: #ff0000, Secondary: #ffff00",...,Richard Pellejero,0.0,0.0,0.0,1.0,0 - 1,Estadio Huancayo,Huancayo,2025-02-16 18:15:00,Micke Palomino
226,13387754,https://www.sofascore.com/deportivo-garcilaso-...,Deportivo Garcilaso,458584,2.0,Comerciantes Unidos,213609,1.0,"Primary: #374df5, Secondary: #374df5","Primary: #ffff00, Secondary: #ffff00",...,Claudio Biaggio,1.0,1.0,1.0,0.0,2 - 1,Estadio Inca Garcilaso de la Vega,Cusco,2025-02-16 20:30:00,Cesar Garcia
227,13387759,https://www.sofascore.com/ayacucho-alianza-uni...,Ayacucho,33894,3.0,Alianza Universidad,306660,2.0,"Primary: #ffffff, Secondary: #000000","Primary: #ff0000, Secondary: #0000c8",...,Roberto Mosquera,2.0,1.0,1.0,1.0,3 - 2,Estadio Ciudad de Cumaná,Ayacucho,2025-02-17 20:00:00,Robin Segura


In [6]:
# Guardar Excel en la carpeta actual
nombre_archivo = f"Datos_totales_{liga}_{year}.xlsx"
df_matches.to_excel(nombre_archivo, index=False, engine='openpyxl')

# Generar enlace de descarga en Jupyter
from IPython.display import FileLink
FileLink(nombre_archivo)

C:\Users\Alvaro\Proyectos\Proyecto Gronestats\GroneStatz\notebooks\Datos_totales_Liga 1 Peru_2025.xlsx